In [1]:
# load systems
import sys
print("Python version : {}". format(sys.version))

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


Python version : 3.6.8 |Anaconda custom (64-bit)| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


Using TensorFlow backend.


# 准备训练数据

In [2]:
train_data = pd.read_csv('./input/train.csv').values
test_data = pd.read_csv('./input/test.csv').values


x_train = train_data[:, 1:].reshape(train_data.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_train /= 255.0

x_test = test_data.reshape(-1, 28, 28, 1)
x_test = x_test.astype('float32')
x_test /= 255.0
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(train_data[:, 0], 10)

print("x_train shape : ", x_train.shape)
print("y_train shape : ", y_train.shape)


x_train shape :  (42000, 28, 28, 1)
y_train shape :  (42000, 10)


# LeNet 代码

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Flatten

class LeNet5:
    """ LeNet-5

        input : 28 * 28
        C1 : 6 feature maps, kernel size 5*5, activation relu
        S2 : 6 feature maps, pooling size 2*2, maxpooling
        C3 : 16 feature maps, kernel size 5*5, activation relu
        S4 : 16 feature maps, pooling size 2*2, maxpooling
        C5 : 120 feature maps, kernel size 5*5, activation relu
        F6 : 84
        output : 10

    """
    @staticmethod
    def build():

        model = Sequential()

        model.add(ZeroPadding2D(padding=(2, 2), input_shape = (28, 28, 1)))

        # C1
        model.add(Conv2D(filters = 6, kernel_size = (5, 5), 
            strides = (1, 1), activation = 'relu' ))

        # S2
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

        # C3
        # TODO:
        model.add(Conv2D(filters = 16, kernel_size = (5, 5),            
            strides = (1, 1), activation = 'relu'))

        # S4
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

        # C5
        model.add(Conv2D(filters = 120, kernel_size = (5, 5),            
            strides = (1, 1), activation = 'relu'))

        model.add(Flatten())

        # F6
        model.add(Dense(units = 84, activation = 'relu'))

        # output
        model.add(Dense(units = 10, activation = 'softmax'))

        model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

        return model

In [4]:
from sklearn.model_selection import train_test_split
epochs = 20

trainX, testX, trainY, testY = train_test_split(x_train, y_train, test_size=0.2)

model = LeNet5.build()

model.fit(trainX, trainY, batch_size=128, epochs=epochs, verbose=1)

(loss, accuracy) = model.evaluate(testX, testY, batch_size=128, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/20
33600/33600 [==============================] - 12s 358us/step - loss: 0.4148 - acc: 0.8660
Epoch 2/20
33600/33600 [==============================] - 12s 349us/step - loss: 0.0932 - acc: 0.9712
Epoch 3/20
33600/33600 [==============================] - 10s 306us/step - loss: 0.0655 - acc: 0.9797
Epoch 4/20
33600/33600 [==============================] - 11s 332us/step - loss: 0.0505 - acc: 0.9838
Epoch 5/20
33600/33600 [==============================] - 10s 311us/step - loss: 0.0420 - acc: 0.9866
Epoch 6/20
33600/33600 [==============================] - 15s 439us/step - loss: 0.0349 - acc: 0.9885
Epoch 7/20
33600/33600 [==============================] - 12s 360us/step - loss: 0.0287 - acc: 0.9904
Epoch 8/20
33600/33600 [==============================] - 12s 360us/step - loss: 0.0241 - acc: 0.9923
Epoch 9/20
33600/33600 [==============================] - 12s 343us/step - loss: 0.0219 - acc: 0.9927
Epoch 10/20
33600/33600 [==============================] - 11s 341us/step - loss: 

In [5]:
results = model.predict(x_test)
labels = np.argmax(results, axis = 1)
labels = pd.Series(labels, name="Label")

In [6]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"), labels],axis = 1)

submission.to_csv("./results/cnn_mnist_datagen.csv",index=False)